<a href="https://colab.research.google.com/github/evbevz/epoxy-supervisor/blob/main/EpoxyLevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.4/904.4 kB 17.4 MB/s eta 0:00:00


In [3]:
from lxml import etree
import os.path
import torch
from ultralytics import YOLO

# Косинусное сходство (Косинус угла между векторами)
def cos_sim (v1, v2):
    cosine_similarity = torch.dot(v1, v2) / (torch.linalg.vector_norm(v1) * torch.linalg.vector_norm(v2))
    return cosine_similarity

# Проекция вектора на вектор
def pr_vec (v1, v2):
    pr_v1_on_v2 = torch.dot(v1, v2) / torch.linalg.vector_norm(v2)
    return pr_v1_on_v2

# Загрузка файла анотации в формате "CVAT for images 1.1"
# Из файла берётся массив точек полилинии с разметкой шкалы деления: 0мл, 1мл, ... 20мл
def Calibrate (fAnnotation):
    tree = etree.parse(fAnnotation)
    root = tree.getroot()
    result = root.xpath('//polyline[@label="injector meter"]')
    if not(result):
        raise Exception("Не удалось найти polyline с точками уровней инжектора в файле: ", fAnnotation)
    points = result[0].attrib['points']
    # Сложная конструкция, но по сути разбирает стороку на список списков (двумерный список) из двух значений попутно переводя из строковых значений в числовое значние.
    list_of_points = list(map(lambda x: list(map(float,x.split(','))), points.split(';')))
    array_of_points = torch.tensor(list_of_points)
    #print ("Массив точек шкалы инжектора:\n", array_of_points)
    return array_of_points

# Получение центра эллипса по точкам: левой и правой большой оси, дальней и ближней малой оси.
# Вычисляем через середину диагоналей эллипса. На будущее стоит просто определять bounding box этого эллипса. Модель будет выдавать уже как раз середину.
def GetEllipseCenter (kpt):
    #print ("Левая точка большой оси:\t", kpt[0])
    #print ("Правая точка большой оси:\t", kpt[1])
    #print ("Дальняя точка малой оси:\t", kpt[2])
    #print ("Ближняя точка малой оси:\t", kpt[3])

    # Проверяем уверенность в определении точки. Если показатель меньше заданного значения, то точку игнорируем.
    # Середина между левой и правой точками большой диагонали эллипса
    if (kpt[0][2]>kptConfidence) and (kpt[1][2]>kptConfidence):
        big_axis_center = (kpt[0][0:2:] + kpt[1][0:2:])/2
    else:
        big_axis_center = None
    # Середина между дальней и ближней точками малой диагонали эллипса
    if (kpt[2][2]>kptConfidence) and (kpt[3][2]>kptConfidence):
        small_axis_center = (kpt[2][0:2:] + kpt[3][0:2:])/2
    else:
        small_axis_center = None
    # Середина между серединой большой и серединой малой диагоналей. Иногда может быть не равная серединам каждой из них (при ошибках в определении ключевых точек моделью).
    if (big_axis_center is not None) and (small_axis_center is not None):
        ellipse_center = (big_axis_center + small_axis_center)/2
    elif (big_axis_center is not None) and (small_axis_center is None):
        ellipse_center = big_axis_center
    elif (big_axis_center is None) and (small_axis_center is not None):
        ellipse_center = small_axis_center
    else:
        ellipse_center = None
    #print ("BAC: ", big_axis_center, "SAC: ", small_axis_center, "ELC:", ellipse_center)
    return ellipse_center

def GetEpoxyLevel (model, arrayEpoxyLevel, file):
    # Запускаем предсказание
    results = model.predict(source=filenameInjectorCam, verbose=False)  # Предсказание по изображению. Возвращается список результатов (т.к. можно передать список кадров или даже видео)
    # Теоретически может быть список результатов, но берём только одно - первое.
    keypoints = results[0].keypoints  # Keypoints object for pose outputs
    #results[0].show()  # display to screen
    #print ('--- Keypoints: ---\n', keypoints)

    # Высчитываем центр эллипса
    # Передаем в параметре 4 точки диагоналей эллипса в виде тензора 4x3 [[x1,y2,confidence1],[]...]
    ellipse_center = GetEllipseCenter(keypoints.data[0][2:6:])
    #ellipse_center = GetEllipseCenter(torch.tensor([[8.2964e+02, 1.1205e+03, 9.8753e-01],
    #    [1.0747e+03, 1.1256e+03, 9.1424e-01],
    #    [9.5690e+02, 1.0506e+03, 9.9983e-01],
    #    [0.0000e+00, 0.0000e+00, 3.4417e-01]], device='cuda:0'))

    if ellipse_center is not None:
        # Переносим массив точек шприца на то же устройство рассчета где и тензоры модели предсказаний. Если расчёты велись на CUDA, то лучше там и считать всё остальное.
        arrayEpoxyLevel = arrayEpoxyLevel.to(ellipse_center.device)

        # Вычисляем ближайшую калиброванную точку к предсказанной точке (середине эллипса)
        LengthMin = (keypoints.orig_shape[0]**2+keypoints.orig_shape[1]**2)**0.5 # Нужно просто большое значение, но решил указать максимально возможное расстояние на изображении (диагональ)
        LevelMin = 0
        for level, kpt in enumerate(arrayEpoxyLevel):
            Length = torch.norm(ellipse_center - kpt) # Расстояние между предсказанной точкой и калиброванной точкой
            if Length < LengthMin: # Если решим, что хоти чтобы при одинаковом расстоянии показывал значение большего уровня, то тогда поставить знак сравнения <=
                LengthMin = Length # Запоминаем минимальное расстояние
                LevelMin = level # Запоминаем уровень к точке которого расстояние минимальное
            print ("Lvl: ", level, "\tCalib pt: ", kpt, "\tPredict pt: ", ellipse_center, "\tLength: ", Length)
        print ("Предсказанный уровень эпоксидки: ", LevelMin)

        # Дорасчёт дробной части уровня.

        vSrc = ellipse_center - arrayEpoxyLevel[LevelMin]
        vNext = arrayEpoxyLevel[LevelMin+1] - arrayEpoxyLevel[LevelMin]
        vPrev = arrayEpoxyLevel[LevelMin-1] - arrayEpoxyLevel[LevelMin]
        print ("vSrc:\t", vSrc, torch.linalg.vector_norm(vSrc))
        print ("vNext:\t", vNext, torch.linalg.vector_norm(vNext))
        print ("vPrev:\t", vPrev, torch.linalg.vector_norm(vPrev))

        cos_next = cos_sim(vSrc,vNext)
        print ('Косинус угла к следующему вектору:', cos_next)
        cos_prev = cos_sim(vSrc,-vPrev)
        print ('Косинус угла к предыдущему вектору:', cos_prev)
        if cos_next > cos_prev:
            pr = pr_vec(vSrc, vNext)
        elif cos_next < cos_prev:
            pr = pr_vec(vSrc, vPrev)
        else:
            pr = 0
        print ("Длина проекции: ", pr)
        return LevelMin
    else:
        return None

# Значение уверенности в правильности распознавания, ниже которого не будем считать, что точки определились правильно. Т.е. координаты такой точки будем считать ложными и точку игнорировать.
kptConfidence = 0.8 # Сейчас точки если и распознаются, то с уверенностью больше 0.9

# Калибровочный файл в котором хранятся уровни от 0 до 20мл. В виде координат X,Y центра эллипса поверхности эпоксидки на каждом уровне.
filenameAnnotation = 'samples/EpoxyLevelCalibrate.annotations.xml'

# Файл весов обученной для распознавания модели
filenameAIModel = 'weights/epoxy-supervisor.20241228.best.pt'

# Изображение для предсказания уровня эпоксидки
#filenameInjectorCam = '/home/nikolay/opencv/epoxy-supervisor/samples/fail01.png'
filenameInjectorCam = 'samples/000019.png'

# Калибруем шприц
arrayEpoxyLevel = Calibrate(filenameAnnotation)

# Загружаем модель
model = YOLO(filenameAIModel)

# Запрос уровня по изображению
GetEpoxyLevel(model, arrayEpoxyLevel, filenameInjectorCam)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Lvl:  0 	Calib pt:  tensor([ 926.0000, 1104.2000]) 	Predict pt:  tensor([1040.8839,  698.4255]) 	Length:  tensor(421.7241)
Lvl:  1 	Calib pt:  tensor([ 937.5000, 1069.4000]) 	Predict pt:  tensor([1040.8839,  698.4255]) 	Length:  tensor(385.1107)
Lvl:  2 	Calib pt:  tensor([ 948.5000, 1034.5000]) 	Predict pt:  tensor([1040.8839,  698.4255]) 	Length:  tensor(348.5410)
Lvl:  3 	Calib pt:  tensor([960.4000, 998.9000]) 	Predict pt:  tensor([1040.8839,  698.4255]) 	Length:  tensor(311.0668)
Lvl:  4 	Calib pt:  tensor([971.2000, 964.0000]) 	Predict pt:  tensor([1040.8839,  698.4255]) 	Length:  tensor(274.5645)
Lvl:  5 	Calib pt:  tensor([983.4000, 927.5000]) 	Predict pt:  tensor([1040.88

10